In [ ]:
import pandas as pd 
import numpy as np
import os

In [ ]:
tuner_output_baseline = '/home/rama.jabal/Melloddy/aux_data/data/Mellody_tuner/all/output_all/images_w_melloddy/'
tuner_input_images = '/home/rama.jabal/Melloddy/aux_data/pseudolabel_auxdata/datapreperation/images_files/input/'



# baseline tuner t8 file 
path_t8 = os.path.join(tuner_output_baseline, 'results_tmp/classification/T8c.csv')
t8 = pd.read_csv(path_t8)


# T0-file csv tuner input creation for the step1 image model
path_t0 = os.path.join(tuner_input_images,'T0.csv')
t0 = pd.read_csv(path_t0)

In [ ]:
t8 = t8[['input_assay_id','threshold','threshold_method']]
t8 = t8.rename(columns={'threshold':'expert_threshold'})

In [ ]:
t8['subgroup_index'] = t8.groupby('input_assay_id').cumcount()+1

In [ ]:
t8

In [ ]:
df = pd.pivot_table(
    t8.drop_duplicates()
    ,columns=['subgroup_index']
    ,values=['expert_threshold']
    ,index=['input_assay_id']
)
df.columns = df.columns.map(lambda x: '_'.join([*map(str, x)]))


In [ ]:
threshold_cols = [f'expert_threshold_{i}' for i in range(1,6)]
t0_out = pd.merge(
    t0.drop(columns=threshold_cols)
    ,df
    ,how='inner'
    ,on='input_assay_id'
)

In [ ]:
# MT will freeze if not all the expert threshold columns are present
for col in threshold_cols :
    if col not in t0_out.columns : 
        t0_out[col] = np.nan

In [ ]:
t0_out.to_csv(os.path.join(tuner_input_images,'T0_synchedthreshold.csv'), index=False)

In [ ]:
iais_in_scope = set(t0_out['input_assay_id'])

In [ ]:
### Aligning also the T1 and T2 files

In [ ]:
path_t1 = os.path.join(tuner_input_images,'T1.csv')
t1 = pd.read_csv(path_t1)

In [ ]:
t1_synchedthreshold = t1.query('input_assay_id in @iais_in_scope')

In [ ]:
t1_synchedthreshold.to_csv(os.path.join(tuner_input_images,'T1_synchedthreshold.csv'), index=False)

In [ ]:
icis = set(t1_synchedthreshold['input_compound_id'])

In [ ]:
path_t2 = os.path.join(tuner_input_images, 'T2.csv')
t2 = pd.read_csv(path_t2)

In [ ]:
t2.query('input_compound_id in @icis').to_csv(os.path.join(tuner_input_images,'T2_synchedthreshold.csv'), index=False)